# Test Suites and Reports for Bicycle Demand Prediction

In [ ]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

from datetime import datetime, time
from sklearn import datasets, ensemble

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, RegressionPreset

from evidently.test_suite import TestSuite
from evidently.test_preset import DataDriftTestPreset, RegressionTestPreset
from evidently.tests import TestValueMeanError, TestValueMAE, TestValueRMSE

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

In [ ]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip", verify=False).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

In [ ]:
raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)

In [ ]:
raw_data.head()

## Regression Model

### Model training

In [ ]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [ ]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [ ]:
reference.head()

In [ ]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [ ]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

In [ ]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
current_prediction = regressor.predict(current[numerical_features + categorical_features])

In [ ]:
reference['prediction'] = ref_prediction
current['prediction'] = current_prediction

### Model Performance 

In [ ]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [ ]:
regression_performance_report = Report(
    metrics=[RegressionPreset()]
)

regression_performance_report.run(reference_data=None, current_data=reference, column_mapping=column_mapping)

In [ ]:
regression_performance_report.show()

In [ ]:
#regression_performance_report.save_html('regression_performance_at_training.html')

##  Week 1

In [ ]:
regression_performance_test = TestSuite(tests=[
    TestValueMeanError(lte=10, gte=-10),
    TestValueMAE(lte=15),
])

regression_performance_test.run(reference_data=None, 
                                current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
                                column_mapping=column_mapping)
regression_performance_test

In [ ]:
regression_performance_test.as_dict()

## Week 2

In [ ]:
regression_performance_test = TestSuite(tests=[
    TestValueMeanError(lte=10, gte=-10),
    TestValueMAE(lte=15),
])

regression_performance_test.run(reference_data=None, 
                                current_data=current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'],
                                column_mapping=column_mapping)
regression_performance_test

## Week 3

In [ ]:
regression_performance_test = TestSuite(tests=[
    TestValueMeanError(lte=10, gte=-10),
    TestValueMAE(lte=15),
])

regression_performance_test.run(reference_data=None, 
                                current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'],
                                column_mapping=column_mapping)
regression_performance_test

## What has happened?

In [ ]:
regression_report = Report(
    metrics=[
        RegressionPreset()
    ]
)

regression_report.run(reference_data=reference, 
                      current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'],
                      column_mapping=column_mapping)
regression_report

In [ ]:
drift_report = Report(
    metrics=[
        DataDriftPreset()
    ]
)

drift_report.run(reference_data=reference, 
                                current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'],
                                column_mapping=column_mapping)
drift_report

# Support Evidently
Enjoyed the tutorial? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently